# Sketching

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_blobs
import random

X = np.random.randint(10,size=20000)
y = np.bincount(X)


## Data Streams

Data streams refer to the continuous flow of data that is generated in real-time.

- Continuous and rapid input of data
- Limited memory to store the data (less than linear in the input size)
- Limited time to process each element
- Sequential access (no random access)
- Algorithms have one (p=1) or very few passes (p={2,3}) over the data

## Morris Counting

- On data streams, we have to reduce memory usage
- Approximate counting uses several such counters, with different head/tail probabilities:
- Set counter to 0
- Update:
• Draw random number x from [0, 1]
• If (x <= 2 -c) c = c + 1
- Query: return 2c - 2
- And it runs in log(log(n)) memory ( E[c] = log(n) .... )

In [ ]:
def morris(X: np.array) -> int:
  counter = 0
  def step(counter: int) -> int:
    ran = random.random()
    if ran <= 1/np.power(2,counter):
      return counter + 1
    return counter
  for _ in X:
    counter = step(counter)
  return np.power(2,counter) - 2
morris(X)

## Reservoir Sampling

- Sampling: selection of a subset of items from a large data set
- Goal: sample retains the properties of the whole data set
- Important for drawing the right conclusions from the data

**Algorithm**

- Sample first m items
- Choose to sample the i’th item (i>m) with probability pi= m/i
- If sampled, randomly replace a previously sampled item
- Optimization: when i gets large, compute which item will be sampled next, skip over intervening items

In [ ]:
def reservoir_sampling(X: np.array, m: int) -> int:
  reservoir = X[:m]
  for i in range(m, len(X)):
    ran = random.random()
    if ran < m/i:
      reservoir[random.randint(0, len(reservoir)-1)] = X[i]
  return reservoir
reservoir_sampling(X, 10)

In [ ]:
def reservoir_sampling_order(X: np.array, k: int) -> int:
  reservoir = []
  for i in X:
    reservoir.append((random.random(), i))
    reservoir = sorted(reservoir)[:k]
  return [i[1] for i in reservoir]
reservoir_sampling_order(X, 10)

## Sketches

- Not every problem can be solved with sampling
• Example: counting how many distinct items in the stream
• If a large fraction of items are not sampled, don’t know if they
are all same or all different
• Other techniques take advantage that the algorithm can
“see” all the data even if it can’t “remember” it all
•
“Sketch”: essentially, a linear transform of the input
• Model stream as defining a vector, sketch is result of multiplying
stream vector by an (implicit) matrix

## FM Sketch (Flajolet-Martin)

- Approach: hash data stream elements uniformly to N bit values, i.e.:
• Task: Given a data stream of, estimate the number of
distinct elements occurring in it (recall Morris counting)
• Assumption: the larger the number of distinct elements in the
stream, the more distinct the occurring hash values, and the
more likely one with an unusual property appears


In [ ]:
class FMSketch:
    def __init__(self, m):
        self.m = m
        self.hash_function = gen_random_hasher(2**m)
        self.maximum = 0

    def add(self, x):
        hashed = str(bin(self.hash_function(x)))[2:].rjust(self.m, '0')
        zeros_at_the_end = (hashed[::-1]+'1').index('1')
        self.maximum = max(self.maximum, zeros_at_the_end)

    def estimate(self):
        return self.maximum**2

X = np.random.randint(128,size=500)
y = np.bincount(X)
fm = FMSketch(32)
for x in X:
    fm.add(x)

estimate = fm.estimate()
print(estimate)

## Majority

- Given a stream of elements, find the element that appears more than half the time

In [ ]:
def majority(X: np.array) -> int:
  c = 0
  v = None
  for i in X:
    if c == 0:
      v = i
      c = 1
    elif v == i:
      c += 1
    else:
      c -= 1
  return v
majority(X)

## Frequent
- Find all elements in a sequence whose frequency exceeds 1/k fraction of a total count (i.e frequency > m/k)

In [ ]:
def frequent(X: np.array, k: int) -> set:
  c = [0 for _ in range(k)]
  T = set()
  for i in X:
    if i in T:
      c[i] += 1
    elif len(T) < k-1:
      T.add(i)
      c[i] += 1
    else:
      c = [j-1 for j in c]
      for j,v in enumerate(c):
        if v == 0:
          T.remove(j)
  return T
frequent(X, 10)

## Bloom Filter
A Bloom filter is a probabilistic data structure used to test whether an element is a member of a set. It works by hashing the element and using the resulting hash to set bits in a bit array. When testing for membership of an element, the same hash function is applied to the element and the corresponding bits in the bit array are checked. If all the corresponding bits are set, the element is considered to be in the set, though there is a small probability of a false positive. Bloom filters are commonly used in situations where the space required for storing a set of elements is a concern, such as in network routers or database systems.

**Algorithm**

- Given:
    * A set of hash functions h1, h2, ..., hn hi: W -> [1, n]
    * A bit vector of size n (initialized to 0)
- to add an element to W
    * for each hash function hi
    * set the bit at position hi(x) to 1
- to test whether an element x is in W
    * compute each hash function hi
    * sum up the returned bits
    * return TRUE if the sum is equal to k, FALSE otherwise

In [ ]:
import secrets

def gen_random_hasher(max_val=1024):
    seed = secrets.randbits(64)
    return lambda val: (hash(val) ^ seed) % max_val

# Hash functions
def get_hash_functions(k: int, maximum: int) -> list:
  return [gen_random_hasher(maximum) for i in range(k)]

In [ ]:
class BloomFilter:
  def __init__(self, k):
    self.hashes = get_hash_functions(k,k)
    self.buckets = [0 for i in range(k)]
    self.k = k
  def push(self, x):
    vals = [i(x) for i in self.hashes]
    for i in vals:
      self.buckets[i]=1
  def query(self, x):
    return np.sum([self.buckets[i(x)] for i in self.hashes]) == self.k

bloom = BloomFilter(10)
bloom.push(5)
print(bloom.query(5))
print(bloom.query(6))
bloom.push(6)
print(bloom.query(6))

## Count-Min Sketch
- Simple sketch idea, can be used for as the basis of many
different stream mining tasks
    * Join aggregates, range queries, moments, ...
    * Creates a small summary as an array of w ́ d in size
    * Use d hash functions to map vector entries to [1..w]

In [ ]:
class FMSketch:
    def __init__(self, m):
        self.m = m
        self.hash_function = gen_random_hasher(2**m)
        self.maximum = 0

    def add(self, x):
        hashed = str(bin(self.hash_function(x)))[2:].rjust(self.m, '0')
        zeros_at_the_end = (hashed[::-1]+'1').index('1')
        self.maximum = max(self.maximum, zeros_at_the_end)

    def estimate(self):
        return self.maximum**2

X = np.random.randint(128,size=500)
y = np.bincount(X)
fm = FMSketch(32)
for x in X:
    fm.add(x)

estimate = fm.estimate()
print(estimate)